<a href="https://colab.research.google.com/github/Dinh-Xuan-Huong/TED/blob/main/TED_youtube_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.makedirs('/content/drive/My Drive/TED/TED_scraping')
os.chdir('/content/drive/My Drive/TED/TED_scraping')
os.makedirs('./data')

In [ ]:
!pip install selenium
!apt-get update 
!apt install -y chromium-chromedriver

     |████████████████████████████████| 911kB 12.8MB/s 
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://d

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

import time
import json
import numpy as np
import operator
from functools import reduce
from multiprocessing import Pool

In [ ]:
def init_driver():
    option = webdriver.ChromeOptions()
    option.add_argument(' — incognito')
    option.add_argument('--headless')
    option.add_argument('--no-sandbox')
    option.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(executable_path='chromedriver', chrome_options=option)
    return (driver)

In [ ]:
def scroll_page(driver, time_sleep=4):
	"""
	Scroll to the bottom of a page.
	"""
	ht=driver.execute_script("return document.documentElement.scrollHeight;")
	while True:
		prev_ht=driver.execute_script("return document.documentElement.scrollHeight;")
		driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
		time.sleep(time_sleep)
		ht=driver.execute_script("return document.documentElement.scrollHeight;")
		if prev_ht==ht:
			break

In [ ]:
def customize_scroll_page(driver, ntimes, time_sleep=5):
    for i in range(ntimes):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(time_sleep)

In [ ]:
def save_json(list_of_dict, path):
	with open(path, 'w') as fout:
		json.dump(list_of_dict , fout)

In [ ]:
def open_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return (data)

In [ ]:
def get_vlinks(channel_url, save=False):
    channel_url_video = channel_url + '/videos'
    driver = init_driver()
    driver.get(channel_url_video)
    scroll_page(driver)
    
    vlink_xpath = './/*[@id="video-title"]'
    class_name = 'style-scope ytd-grid-video-renderer'
    
    videos = driver.find_elements_by_class_name(class_name)
    vlink_list = list(map(lambda video: video.find_element_by_xpath(vlink_xpath).get_attribute('href'), videos))

    if save:
        save_json(vlink_list, './data/all_video_links.json')

    return (link_list)

In [ ]:
def get_video_info(driver, url_video):
    driver.get(url_video)
    customize_scroll_page(driver=driver, ntimes=3)

    title_xpath = './/*[@id="container"]/h1/yt-formatted-string'
    public_date_xpath = './/*[@id="date"]/yt-formatted-string'
    event_xpath = './/*[@id="text"]/a'
    nview_xpath = './/*[@id="count"]/yt-view-count-renderer/span[1]'
    nlike_xpath = '(//yt-formatted-string[@class="style-scope ytd-toggle-button-renderer style-text"])[1]'
    ndislike_xpath = '(//yt-formatted-string[@class="style-scope ytd-toggle-button-renderer style-text"])[2]'
    ncomment_xpath = './/*[@id="count"]/yt-formatted-string'

    def handle_off_comment():
        try:
            return (driver.find_element_by_xpath(ncomment_xpath).text)
        except NoSuchElementException:
            return ("Off")
    
    return (
        {
            "title": driver.find_element_by_xpath(title_xpath).text,
            "yid": url_video.split("v=")[1],
            "puplic_date": driver.find_element_by_xpath(public_date_xpath).text,
            "event": driver.find_element_by_xpath(event_xpath).text,
            "nview": driver.find_element_by_xpath(nview_xpath).text,
            "nlike": driver.find_element_by_xpath(nlike_xpath).get_attribute('aria-label'),
            "ndislike": driver.find_element_by_xpath(ndislike_xpath).get_attribute('aria-label'),
            "ncomment": handle_off_comment()
        }
    )

In [ ]:
def get_vinfo(url_video):
  """
  Scrape TED video info from youtube. 
  Log out the scraping status, whether success or fail.
  """
  log_path = './LOG.txt'

  driver = init_driver()
  with open(log_path, "a") as f:
    try:
      res = get_video_info(driver, url_video, vid)
      f.write(f"{str(vid)} \t\t {url_video}\n")
    except NoSuchElementException:
      f.write(f"{str(vid)} \t\t {url_video} \t\t FAIL\n")
    else:
      return(res)
    finally:
      driver.quit()

# Get all video urls

In [ ]:
if __name__ == "__main__":

    # Create an empty json file which is info_of_TED_youtube_videos.json to store the result
    save_json([], './data/info_of_TED_youtube_videos.json')

    # Get all videos of TED channel from video tab
    ted_url = 'https://www.youtube.com/c/TED'
    need_update_vlinks = get_vlinks(ted_url, save=True)

    # need_update_vlinks = open_json('./data/all_video_links.json') # Use it in case you already have this list
    total_videos = len(need_update_vlinks)

    res_list = open_json('./data/info_of_TED_youtube_videos.json')

    start_index, end_index = 0, 10
    start_threshold = (total_videos//10)*10
    while start_index <= total_videos:
      try:
        pool = Pool()
        data_output = pool.map(get_vinfo, need_update_vlinks[start_index:end_index])
      finally:
        pool.close()
        pool.join()

      res_list = res_list + data_output
      save_json(res_list, './data/info_of_TED_youtube_videos.json')
      
      start_index = start_index + 10
      end_index = total_videos if start_index == start_threshold else end_index + 10
        
      print(start_index, end_index)
